<h1><center> Credit Card Fraud Detection </center></h1>
<h2><center> Part 2. Classification Models </center></h2>
<h2><center> Sugata Ghosh and Shyambhu Mukherjee </center></h2>

# Contents

- [Introduction](#1.-Introduction)
- [Evaluation Metrics](#2.-Evaluation-Metrics)
- [Train-Test Split](#3.-Train-Test-Split)
- [Feature Scaling](#4.-Feature-Scaling)
- [Logistic Regression](#5.-Logistic-Regression)
- [$k$-Nearest Neighbors ($k$-NN)](#6.-KNN)
- [Decision Tree](#7.-Decision-Tree)
- [Support Vector Machine (SVM)](#8.-SVM)
- [Naive Bayes](#9.-Naive-Bayes)
- [Random Forest](#10.-Random-Forest)
- [Linear Discriminant Analysis (LDA)](#11.-LDA)
- [Stochastic Gradient Descent (SGD)](#12.-SGD)
- [Ridge Classifier](#13.-Ridge-Classifier)
- [Conclusion](#14.-Conclusion)

<a name='1.-Introduction'></a>
# 1. Introduction

In [ ]:
# Importing necessary libraries

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import shutil
columns = shutil.get_terminal_size().columns

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import average_precision_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier

import imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

from IPython.display import Javascript
from IPython.display import display

## Data

Source: https://www.kaggle.com/mlg-ulb/creditcardfraud

The dataset contains information on the transactions made using credit cards by European cardholders, in two particular days of September $2013$. It presents a total of $284807$ transactions, of which $492$ were fraudulent. Clearly, the dataset is highly imbalanced, the positive class (fraudulent transactions) accounting for only $0.173\%$ of all transactions.

For a particular transaction, the feature **Time** represents the time (in seconds) elapsed between the transaction and the very first transaction, **Amount** represents the amount of the transaction and **Class** represents the status of the transaction with respect to authenticity. The class of an authentic (resp. fraudulent) transaction is taken to be $0$ (resp. $1$). Rest of the variables (**V1** to **V28**) are obtained from principle component analysis (PCA) transformation on original features that are not available due to confidentiality.

In [ ]:
# The dataset

data = pd.read_csv('../input/creditcardfraud/creditcard.csv')
data

In [ ]:
features = list(data.columns)
features.remove('Class')

In [ ]:
# Statistical descriptions of the features

features_stat = data.drop(['Class'], axis = 1)
features_stat.describe()

## Objectives of the project

### Primary objective:

**Classification of transactions as authentic or fraudulent**. To be prcise, given the data on **Time**, **Amount** and transformed features **V1** to **V28** for a particular transaction, our goal is to correctly classify the transaction as **authentic** or **fraudulent**. We employ different techniques to build classification models and compare them by various evaluation metrics.

### Secondary objectives:

Answering the following questions using machine learning and statistical tools and techniques.

- When a fraudulent transaction is made, is it followed soon by one or more such fraudulent transactions? In other words, do the attackers make consecutive fraudulent transactions in a short span of time?


- Is the amount of a fraudulent transaction generally larger than that of an authentic transaction?


- Is there any indication in the data that fraudulent transactions occur at high-transaction period?


- It is seen from the data that the number of transactions are high in some time intervals and low in between. Does the occurance of frauds related to these time intervals?


- There are a few time-points which exhibits high number of fraud transactions. Is it due to high number of total transactions or due to some other reason?

**In this part we shall classify transactions as authentic or fraudulent based on the information available on independent features (time, amount and the transformed variables V1-V28). One issue with the dataset is that it is highly imbalanced in terms of the target variable Class. Thus we run into the risk of training the models with a representative sample of fraudulent transactions of extremely small size. We employ different approaches to deal with this problem. The performance of each model is checked through various evaluation metrics and is summarized in tabulated form.**

<a name='2.-Evaluation-Metrics'></a>
# 2. Evaluation Metrics

Any prediction about a binary categorical target variable  falls into one of the four categories:
- True Positive: The classification model correctly predicts the output to be positive
- True Negative: The classification model correctly predicts the output to be negative
- False Positive: The classification model incorrectly predicts the output to be positive
- False Negative: The classification model incorrectly predicts the output to be negative

| $\downarrow$ Actual state / Predicted state $\rightarrow$ | Positive | Negative |
| :---: | :---: | :---: |
| Positive | True Positive | False Negative |
| Negative | False Positive | True Negative |

Let **TP**, **TN**, **FP** and **FN** respectively denote the number of **true positives**, **true negatives**, **false positives** and **false negatives** among the predictions made by a particular classification model. Below we give the definitions of some evaluation metrics based on these four quantities.

$$\text{Accuracy} = \frac{\text{Number of correct predictions}}{\text{Number of total predictions}} = \frac{TP + TN}{TP + TN + FP + FN}$$

- **Precision-Recall Metrics**

\begin{align*}
&\text{Precision} = \frac{\text{Number of true positive predictions}}{\text{Number of total positive predictions}} = \frac{TP}{TP + FP}\\\\
&\text{Recall} = \frac{\text{Number of true positive predictions}}{\text{Number of total positive cases}} = \frac{TP}{TP + FN}\\\\
&\text{Fowlkes-Mallows index (FM)} = \text{Geometric mean of Precision and Recall} = \sqrt{\text{Precision} \times \text{Recall}}\\\\
&F_1\text{-Score} = \text{Harmonic mean of Precision and Recall} = \frac{2 \times \text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}\\\\
&F_{\beta}\text{-Score} = \frac{\left(1 + \beta^2\right) \times \text{Precision} \times \text{Recall}}{\left(\beta^2 \times \text{Precision}\right) + \text{Recall}},
\end{align*}

where $\beta$ is a positive factor, chosen such that Recall is $\beta$ times as important as Precision in the analysis. Popular choices of $\beta$ are $0.5$, $1$ and $2$.

- **Sensitivity-Specificity Metrics**

\begin{align*}
&\text{Sensitivity} = \frac{\text{Number of true positive predictions}}{\text{Number of total positive cases}} = \frac{TP}{TP + FN}\\\\
&\text{Specificity} = \frac{\text{Number of true negative predictions}}{\text{Number of total negative cases}} = \frac{TN}{TN + FP}\\\\
&\text{G-mean} = \text{Geometric mean of Sensitivity and Specificity} = \sqrt{\text{Sensitivity} \times \text{Specificity}}
\end{align*}

- **Area Under Curve (AUC) Metrics**

Consider the following quantities:

\begin{align*}
&\text{True Positive Rate (TPR)} = \frac{\text{Number of true positive predictions}}{\text{Number of total positive cases}} = \frac{TP}{TP + FN}\\\\
&\text{False Positive Rate (FPR)} = \frac{\text{Number of false positive predictions}}{\text{Number of total negative cases}} = \frac{FP}{FP + TN}
\end{align*}

The Receiver Operating Characteristic (ROC) curve is obtained by plotting TPR against FPR for a number of threshold probability values. The area under ROC curve (ROC-AUC) serves as a valid evaluation metric.

Similarly, the Precision-Recall (PR) curve is obtained by plotting Precision against Recall for a number of threshold probability values. The area under PR curve (PR-AUC) is also a valid evaluation metric. Another widely used metric in this regard is the Average Precision (AP), which is a weighted mean of precisions at each threshold, with the weights being increase in recall from the previous threshold.

- **Other Metrics**

\begin{align*}
&\text{Matthews Correlation Coefficient (MCC)} = \frac{\left(TP \times TN\right) - \left(FP \times FN\right)}{\sqrt{\left(TP + FP\right) \times \left(TP + FN\right) \times \left(TN + FP\right) \times \left(TN + FN\right)}}
\end{align*}

Unlike the previous metrics, **MCC** varies from $-1$ (worst case scenario) to $1$ (best case scenario: perfect prediction).

Note that **Recall** and **Sensitivity** are essentially the same quantity.

Among the discussed metrics, some good choices to evaluate models, in particular for imbalanced dataset are **MCC** and **$F_1$-Score**, while **Precision** and **Recall** also give useful information. We shall not give much importance to the **Accuracy** metric in this project as it produces misleading conclusion when the classes are not balanced. In the problem at hand, false negative (a fraudulent transaction being classified as authentic) is more dangerous than false positive (an authentic transaction being classified as fraudulent) as in the former case, the fraudster can cause further financial damage, while in the latter case the bank can cross-verify the authenticity of the transaction from the card-user after taking necessary steps to secure the card. Considering this fact, we give **$F_2$-Score** special importance in evaluating the models.

In [ ]:
EvalMetricLabels = ['MCC', 'F1-Score', 'F2-Score', 'Recall', 'Precision',
                    'FM index', 'Specificity', 'G-mean', 'F0.5-Score', 'Accuracy']

<a name='3.-Train-Test-Split'></a>
# 3. Train-Test Split

## Splitting the data into training set and testing set

In [ ]:
# Separating independent variables and target variable

y = data['Class'] # target variable
X = data.drop('Class', axis = 1) # independent variables

# Constructing training set and testing set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 25)

In [ ]:
z = list(y_train)

count0 = 0
count1 = 0

for i in z:
    if i == 0:
        count0 = count0 + 1
    elif i == 1:
        count1 = count1 + 1

# Class frequencies

class_label_train = ['Authentic', 'Fraud']
class_frequency_train = [count0, count1]

fig1 = px.pie(values = class_frequency_train,
             names = class_label_train,
             title = 'Frequency comparison of authentic and fraudulent transactions in the training dataset',
             template = 'ggplot2'
            )
fig1.show()

The plot indicates extremely high imbalance between the two classes in the training set which may lead to producing misleading models.

## Balancing the training set

### Separating the training set by class

In [ ]:
data_train = pd.concat([X_train, y_train], axis = 1)
data_train_authentic = data_train[data_train['Class'] == 0]
data_train_fraudulent = data_train[data_train['Class'] == 1]

In [ ]:
# Amount vs Time for authentic and fraudulent transactions in the training set

class_list_train = list(y_train)
fraud_status_train = []
for i in range(len(class_list_train)):
    fraud_status_train.append(bool(class_list_train[i]))

fig1 = px.scatter(data_train,
                 x = 'Time',
                 y = 'Amount', 
                 facet_col = fraud_status_train,
                 color = fraud_status_train,
                 title = 'Amount vs Time for the training set',
                 template = 'ggplot2'
                )
fig1.show()

### Random under-sampling (RUS)

We take a subset of the majority class to balance the training dataset.

**Advantage:** Improves run-time and solves any storage issue due to large learning dataset.

**Disadvantage:** Ignores a chunk of information that could be impactful in the analysis and uses only a sample representative of the majority class which is not guarunteed to reflect the same accurately.

In [ ]:
data_train_authentic_under = data_train_authentic.sample(len(data_train_fraudulent))
data_train_under = pd.concat([data_train_authentic_under, data_train_fraudulent], axis = 0)

X_train_under = data_train_under.drop('Class', axis = 1)
y_train_under = data_train_under['Class']

print('Class frequencies after under-sampling:')
print(y_train_under.value_counts())
y_train_under.value_counts().plot(kind = 'bar', title = 'Class frequencies after under-sampling')

In [ ]:
# Amount vs Time for authentic and fraudulent transactions in the training set after random under-sampling

class_list = list(y_train_under)
fraud_status = []
for i in range(len(class_list)):
    fraud_status.append(bool(class_list[i]))

fig1 = px.scatter(data_train_under,
                 x = 'Time',
                 y = 'Amount', 
                 facet_col = fraud_status,
                 color = fraud_status,
                 title = 'Amount vs Time for the training set after random under-sampling',
                 template = 'ggplot2'
                )
fig1.show()

Comparing with the same plots for full dataset, we see that high-amount authentic transactions are not represented in the sample taken from the majority class. This indicates a general drawback of the under-sampling techniques which throws away a major chunk of information from the majority class and suffers from not representing the same accurately.

### Random over-sampling (ROS)

In [ ]:
data_train_fraudulent_over = data_train_fraudulent.sample(len(data_train_authentic), replace = 'True')
data_train_over = pd.concat([data_train_authentic, data_train_fraudulent_over], axis = 0)

X_train_over = data_train_over.drop('Class', axis = 1)
y_train_over = data_train_over['Class']

print('Class frequencies after over-sampling:')
print(y_train_over.value_counts())
y_train_over.value_counts().plot(kind = 'bar', title = 'Class frequencies after over-sampling')

In [ ]:
# Amount vs Time for authentic and fraudulent transactions in the training set after random over-sampling

class_list = list(y_train_over)
fraud_status = []
for i in range(len(class_list)):
    fraud_status.append(bool(class_list[i]))

fig1 = px.scatter(data_train_over,
                 x = 'Time',
                 y = 'Amount', 
                 facet_col = fraud_status,
                 color = fraud_status,
                 title = 'Amount vs Time',
                 template = 'ggplot2'
                )
fig1.show()

These plots resemble the corresponding *Amount vs Time* plots for the full dataset much more accurately than the corresponding plots for the training set obtained from random under-sampling.

### Random under-sampling with imbalanced-learn library (RUS-IL)

In [ ]:
imblearn_rus = RandomUnderSampler(random_state = 40, replacement = True)
X_train_rus, y_train_rus = imblearn_rus.fit_resample(X_train, y_train)

X_train_rus = pd.DataFrame(X_train_rus)
X_train_rus.columns = features

y_train_rus = pd.DataFrame(y_train_rus)
y_train_rus.columns = ['Class']

data_train_under_imblearn = pd.concat([X_train_rus, y_train_rus], axis = 1)

X_train_under_imblearn = data_train_under_imblearn.drop('Class', axis = 1)
y_train_under_imblearn = data_train_under_imblearn['Class']

print('Class frequencies after under-sampling via imbalanced-learn library:')
print(y_train_under_imblearn.value_counts())
y_train_under_imblearn.value_counts().plot(kind = 'bar',
                                           title = 'Class frequencies after under-sampling via imbalanced-learn library')

In [ ]:
# Amount vs Time for authentic and fraudulent transactions in the training set after random under-sampling via imblearn

class_list = list(y_train_under_imblearn)
fraud_status = []
for i in range(len(class_list)):
    fraud_status.append(bool(class_list[i]))

fig1 = px.scatter(data_train_under_imblearn,
                 x = 'Time',
                 y = 'Amount', 
                 facet_col = fraud_status,
                 color = fraud_status,
                 title = 'Amount vs Time',
                 template = 'ggplot2'
                )
fig1.show()

### Random over-sampling with imbalanced-learn library (ROS-IL)

In [ ]:
imblearn_ros = RandomOverSampler(random_state = 40)
X_train_ros, y_train_ros = imblearn_ros.fit_resample(X_train, y_train)

X_train_ros = pd.DataFrame(X_train_ros)
X_train_ros.columns = features

y_train_ros = pd.DataFrame(y_train_ros)
y_train_ros.columns = ['Class']

data_train_over_imblearn = pd.concat([X_train_ros, y_train_ros], axis = 1)

X_train_over_imblearn = data_train_over_imblearn.drop('Class', axis = 1)
y_train_over_imblearn = data_train_over_imblearn['Class']

print('Class frequencies after over-sampling via imbalanced-learn library:')
print(y_train_over_imblearn.value_counts())
y_train_over_imblearn.value_counts().plot(kind = 'bar',
                                          title = 'Class frequencies after over-sampling via imbalanced-learn library')

In [ ]:
# Amount vs Time for authentic and fraudulent transactions in the training set after random over-sampling via imblearn

class_list = list(y_train_over_imblearn)
fraud_status = []
for i in range(len(class_list)):
    fraud_status.append(bool(class_list[i]))

fig1 = px.scatter(data_train_over_imblearn,
                 x = 'Time',
                 y = 'Amount', 
                 facet_col = fraud_status,
                 color = fraud_status,
                 title = 'Amount vs Time',
                 template = 'ggplot2'
                )
fig1.show()

### Synthetic minority over-sampling technique (SMOTE)

In [ ]:
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

X_train_smote = pd.DataFrame(X_train_smote)
X_train_smote.columns = features
X_train_smote

y_train_smote = pd.DataFrame(y_train_smote)
y_train_smote.columns = ['Class']
y_train_smote

data_train_over_smote = pd.concat([X_train_smote, y_train_smote], axis = 1)

X_train_over_smote = data_train_over_smote.drop('Class', axis = 1)
y_train_over_smote = data_train_over_smote['Class']

print('Class frequencies after over-sampling via SMOTE:')
print(y_train_over_smote.value_counts())
y_train_over_smote.value_counts().plot(kind = 'bar', title = 'Class frequencies after over-sampling via SMOTE')

In [ ]:
# Amount vs Time for authentic and fraudulent transactions in the training set after over-sampling via SMOTE

class_list = list(y_train_over_smote)
fraud_status = []
for i in range(len(class_list)):
    fraud_status.append(bool(class_list[i]))

fig1 = px.scatter(data_train_over_smote,
                 x = 'Time',
                 y = 'Amount', 
                 facet_col = fraud_status,
                 color = fraud_status,
                 title = 'Amount vs Time',
                 template = 'ggplot2'
                )
fig1.show()

### Under-sampling via NearMiss (NM)

In [ ]:
nm = NearMiss()
X_train_nm, y_train_nm = nm.fit_resample(X_train, y_train)

X_train_nm = pd.DataFrame(X_train_nm)
X_train_nm.columns = features
X_train_nm

y_train_nm = pd.DataFrame(y_train_nm)
y_train_nm.columns = ['Class']
y_train_nm

data_train_under_nm = pd.concat([X_train_nm, y_train_nm], axis = 1)

X_train_under_nm = data_train_under_nm.drop('Class', axis = 1)
y_train_under_nm = data_train_under_nm['Class']

print('Class frequencies after under-sampling via NearMiss:')
print(y_train_under_nm.value_counts())
y_train_under_nm.value_counts().plot(kind = 'bar', title = 'Class frequencies after under-sampling via NearMiss')

In [ ]:
# Amount vs Time for authentic and fraudulent transactions in the training set after random under-sampling via NearMiss

class_list = list(y_train_under_nm)
fraud_status = []
for i in range(len(class_list)):
    fraud_status.append(bool(class_list[i]))

fig1 = px.scatter(data_train_under_nm,
                 x = 'Time',
                 y = 'Amount', 
                 facet_col = fraud_status,
                 color = fraud_status,
                 title = 'Amount vs Time',
                 template = 'ggplot2'
                )
fig1.show()

From the left plot it is clear that the majority class is not represented accurately in the under-sampling scheme via NearMiss.

In [ ]:
TrainingSets = ['Unaltered', 'RUS', 'ROS', 'RUS-IL', 'ROS-IL', 'SMOTE', 'NM']

<a name='4.-Feature-Scaling'></a>
# 4. Feature Scaling

It may be natural for one of the features to contribute to the classification process more than another. But often this is caused artificially by the difference of range of values that the features take (often due to the units in which the features are measured). Many algorithms, especially the tree-based ones like decision tree and random forest, as well as graphical model-based classifiers like linear discriminant analysis and naive Bayes are invariant to scaling and hence are indifferent to feature scaling. On the other hand, the algorithms based on distances or similarities, which include $k$-nearest neighbours, support vector machine and stochastic gradient descent are sensitive to scaling. This necessitates the practitioner to scale the features appropriately before feeding the data to such classifiers.

In [ ]:
scaling = MinMaxScaler(feature_range = (-1,1)).fit(X_train)

X_train_scaled_minmax = scaling.transform(X_train)
X_train_under_scaled_minmax = scaling.transform(X_train_under)
X_train_over_scaled_minmax = scaling.transform(X_train_over)
X_train_under_imblearn_scaled_minmax = scaling.transform(X_train_under_imblearn)
X_train_over_imblearn_scaled_minmax = scaling.transform(X_train_over_imblearn)
X_train_over_smote_scaled_minmax = scaling.transform(X_train_over_smote)
X_train_under_nm_scaled_minmax = scaling.transform(X_train_under_nm)

X_test_scaled_minmax = scaling.transform(X_test)

<a name='5.-Logistic-Regression'></a>
# 5. Logistic Regression

In [ ]:
logreg = LogisticRegression(max_iter = 1000)

In [ ]:
# Computation of confusion matrix, evaluation metrics and visualization of classes

def classification(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    y_test = list(y_test)
    y_pred = list(y_pred)

    # Confusion matrix
    
    class_names = ['Authentic', 'Fraudulent']
    tick_marks_y = [0.25, 1.2]
    tick_marks_x = [0.5, 1.5]

    confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
    confusion_matrix_df = pd.DataFrame(confusion_matrix, range(2), range(2))
    plt.figure(figsize = (6, 4.75))
    sns.set(font_scale = 1.4) # label size
    plt.title("Confusion Matrix")
    sns.heatmap(confusion_matrix_df, annot = True, annot_kws = {"size": 16}, fmt = 'd') # font size
    plt.yticks(tick_marks_y, class_names, rotation = 'vertical')
    plt.xticks(tick_marks_x, class_names, rotation = 'horizontal')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.grid(False)
    plt.show()

    # Evaluation metrics

    TN = confusion_matrix[0, 0]
    FP = confusion_matrix[0, 1]
    FN = confusion_matrix[1, 0]
    TP = confusion_matrix[1, 1]

    accuracy = (TP + TN)/(TP + FN + TN + FP)
    
    if (FP + TP == 0):
        precision = float('NaN')
    else:
        precision = TP/(TP + FP)
        
    if (TP + FN == 0):
        recall = float('NaN')
    else:
        recall = TP/(TP + FN)
    
    FM_index = np.sqrt(precision * recall) # Fowlkes-Mallows index

    if (TP == 0):
        F0_5_score = float('NaN')
        F1_score = float('NaN')
        F2_score = float('NaN')
    else:
        F0_5_score = (1.25 * precision * recall)/((0.25 * precision) + recall)
        F1_score = (2 * precision * recall)/(precision + recall)
        F2_score = (5 * precision * recall)/((4 * precision) + recall)
    
    if (TN + FP == 0):
        specificity = float('NaN')
    else:
        specificity = TN/(TN + FP)

    G_mean = np.sqrt(recall * specificity)

    MCC_num = (TN * TP) - (FP * FN)
    MCC_denom = np.sqrt((FP + TP) * (FN + TP) * (TN + FP) * (TN + FN))
    
    if (MCC_denom == 0):
        MCC = float('NaN')
    else:
        MCC = MCC_num / MCC_denom # Matthews Correlation Coefficient
    
    # Summary

    EvalMetricLabels = ['MCC', 'F1-Score', 'F2-Score', 'Recall', 'Precision',
                        'FM index', 'Specificity', 'G-mean', 'F0.5-Score', 'Accuracy']
    EvalMetricValues = [MCC, F1_score, F2_score, recall, precision, FM_index, specificity, G_mean, F0_5_score, accuracy]
    
    global summary
    summary = pd.DataFrame(columns = ['Metric', 'Performance score'])
    summary['Metric'] = EvalMetricLabels
    summary['Performance score'] = EvalMetricValues
    
    # Performance of the model through confusion matrix
    
    fig1 = make_subplots(rows = 1, cols = 2, specs = [[{"type": "pie"}, {"type": "pie"}]])

    fig1.add_trace(go.Pie(
        labels = ['TP', 'FN'],
        values = [TP, FN],
        domain = dict(x = [0, 0.4]),
        name = 'Positive Class'), 
        row = 1, col = 1)

    fig1.add_trace(go.Pie(
        labels = ['TN', 'FP'],
        values = [TN, FP],
        domain = dict(x = [0.4, 0.8]),
        name = 'Negative Class'),
        row = 1, col = 2)

    fig1.update_layout(height = 450, showlegend = True)
    fig1.show()

## Unaltered training set

In [ ]:
# Elements of confusion matrix

classification(logreg, X_train, y_train, X_test, y_test)

# Summary of evaluation metrics

summary_logreg_unaltered = summary.copy()
summary_logreg_unaltered.set_index('Metric')

y_score = logreg.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = logreg.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_logreg_unaltered_extended = summary.copy()
summary_logreg_unaltered_extended.loc[len(summary_logreg_unaltered_extended.index)] = ['AP', average_precision]
summary_logreg_unaltered_extended.loc[len(summary_logreg_unaltered_extended.index)] = ['ROC-AUC', roc_auc]
summary_logreg_unaltered_extended.set_index('Metric')

summary_logreg_unaltered_index = summary_logreg_unaltered_extended.T
summary_logreg_unaltered_index.columns = summary_logreg_unaltered_index.iloc[0]
summary_logreg_unaltered_index.drop(summary_logreg_unaltered_index.index[0], inplace = True)
summary_logreg_unaltered_index

**Observation:** While logistic regression model on unaltered training set performs exceedingly well on the negative class (authentic transactions), it does not work so well with the critical positive class (fraudulent transactions) as it misclassifies more than one-third of the transactions in that class.

## Random under-sampling

In [ ]:
# Elements of confusion matrix

classification(logreg, X_train_under, y_train_under, X_test, y_test)

# Summary of evaluation metrics

summary_logreg_under = summary
summary_logreg_under.set_index('Metric')

y_score = logreg.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = logreg.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_logreg_under_extended = summary.copy()
summary_logreg_under_extended.loc[len(summary_logreg_under_extended.index)] = ['AP', average_precision]
summary_logreg_under_extended.loc[len(summary_logreg_under_extended.index)] = ['ROC-AUC', roc_auc]
summary_logreg_under_extended.set_index('Metric')

summary_logreg_under_index = summary_logreg_under_extended.T
summary_logreg_under_index.columns = summary_logreg_under_index.iloc[0]
summary_logreg_under_index.drop(summary_logreg_under_index.index[0], inplace = True)
summary_logreg_under_index

## Random over-sampling

In [ ]:
# Elements of confusion matrix

classification(logreg, X_train_over, y_train_over, X_test, y_test)

# Summary of evaluation metrics

summary_logreg_over = summary
summary_logreg_over.set_index('Metric')

y_score = logreg.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = logreg.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_logreg_over_extended = summary.copy()
summary_logreg_over_extended.loc[len(summary_logreg_over_extended.index)] = ['AP', average_precision]
summary_logreg_over_extended.loc[len(summary_logreg_over_extended.index)] = ['ROC-AUC', roc_auc]
summary_logreg_over_extended.set_index('Metric')

summary_logreg_over_index = summary_logreg_over_extended.T
summary_logreg_over_index.columns = summary_logreg_over_index.iloc[0]
summary_logreg_over_index.drop(summary_logreg_over_index.index[0], inplace = True)
summary_logreg_over_index

## Random under-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(logreg, X_train_under_imblearn, y_train_under_imblearn, X_test, y_test)

# Summary of evaluation metrics

summary_logreg_under_imblearn = summary
summary_logreg_under_imblearn.set_index('Metric')

y_score = logreg.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = logreg.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_logreg_under_imblearn_extended = summary.copy()
summary_logreg_under_imblearn_extended.loc[len(summary_logreg_under_imblearn_extended.index)] = ['AP', average_precision]
summary_logreg_under_imblearn_extended.loc[len(summary_logreg_under_imblearn_extended.index)] = ['ROC-AUC', roc_auc]
summary_logreg_under_imblearn_extended.set_index('Metric')

summary_logreg_under_imblearn_index = summary_logreg_under_imblearn_extended.T
summary_logreg_under_imblearn_index.columns = summary_logreg_under_imblearn_index.iloc[0]
summary_logreg_under_imblearn_index.drop(summary_logreg_under_imblearn_index.index[0], inplace = True)
summary_logreg_under_imblearn_index

## Random over-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(logreg, X_train_over_imblearn, y_train_over_imblearn, X_test, y_test)

# Summary of evaluation metrics

summary_logreg_over_imblearn = summary
summary_logreg_over_imblearn.set_index('Metric')

y_score = logreg.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = logreg.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_logreg_over_imblearn_extended = summary.copy()
summary_logreg_over_imblearn_extended.loc[len(summary_logreg_over_imblearn_extended.index)] = ['AP', average_precision]
summary_logreg_over_imblearn_extended.loc[len(summary_logreg_over_imblearn_extended.index)] = ['ROC-AUC', roc_auc]
summary_logreg_over_imblearn_extended.set_index('Metric')

summary_logreg_over_imblearn_index = summary_logreg_over_imblearn_extended.T
summary_logreg_over_imblearn_index.columns = summary_logreg_over_imblearn_index.iloc[0]
summary_logreg_over_imblearn_index.drop(summary_logreg_over_imblearn_index.index[0], inplace = True)
summary_logreg_over_imblearn_index

## Synthetic minority over-sampling technique (SMOTE)

In [ ]:
# Elements of confusion matrix

classification(logreg, X_train_over_smote, y_train_over_smote, X_test, y_test)

# Summary of evaluation metrics

summary_logreg_over_smote = summary
summary_logreg_over_smote.set_index('Metric')

y_score = logreg.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = logreg.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_logreg_over_smote_extended = summary.copy()
summary_logreg_over_smote_extended.loc[len(summary_logreg_over_smote_extended.index)] = ['AP', average_precision]
summary_logreg_over_smote_extended.loc[len(summary_logreg_over_smote_extended.index)] = ['ROC-AUC', roc_auc]
summary_logreg_over_smote_extended.set_index('Metric')

summary_logreg_over_smote_index = summary_logreg_over_smote_extended.T
summary_logreg_over_smote_index.columns = summary_logreg_over_smote_index.iloc[0]
summary_logreg_over_smote_index.drop(summary_logreg_over_smote_index.index[0], inplace = True)
summary_logreg_over_smote_index

## Under-sampling via NearMiss

In [ ]:
# Elements of confusion matrix

classification(logreg, X_train_under_nm, y_train_under_nm, X_test, y_test)

# Summary of evaluation metrics

summary_logreg_under_nm = summary
summary_logreg_under_nm.set_index('Metric')

y_score = logreg.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = logreg.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_logreg_under_nm_extended = summary.copy()
summary_logreg_under_nm_extended.loc[len(summary_logreg_under_nm_extended.index)] = ['AP', average_precision]
summary_logreg_under_nm_extended.loc[len(summary_logreg_under_nm_extended.index)] = ['ROC-AUC', roc_auc]
summary_logreg_under_nm_extended.set_index('Metric')

summary_logreg_under_nm_index = summary_logreg_under_nm_extended.T
summary_logreg_under_nm_index.columns = summary_logreg_under_nm_index.iloc[0]
summary_logreg_under_nm_index.drop(summary_logreg_under_nm_index.index[0], inplace = True)
summary_logreg_under_nm_index

## Summary of logistic regression models

Keeping in mind that the dataset is highly imbalanced and that the positive class (fraudulent transactions) is more important than the negative class (authentic transactions), we report **MCC**, **F1-Score**, **F2-Score** and **Recall** for each model considered. Additionally we report **Precision**, **FM index**, **Accuracy** and **Specificity**.

In [ ]:
summary_logreg = pd.DataFrame(columns = ['Metric'])

summary_logreg['Metric'] = EvalMetricLabels
summary_logreg_list = [summary_logreg_unaltered, summary_logreg_under, summary_logreg_over, summary_logreg_under_imblearn,
                       summary_logreg_over_imblearn, summary_logreg_over_smote, summary_logreg_under_nm]

for i in summary_logreg_list:
    summary_logreg = pd.merge(summary_logreg, i, on = 'Metric')
    
TrainingSetsMetric = TrainingSets.copy()
TrainingSetsMetric.insert(0, 'Metric')

summary_logreg.columns = TrainingSetsMetric
summary_logreg.set_index('Metric', inplace = True)
summary_logreg

In [ ]:
# Function to visually compare performances of the model applied on different training sets through evaluation metrics

def summary_visual(summary_model):
  fig1 = make_subplots(rows = 2, cols = 4, shared_yaxes = True, subplot_titles = EvalMetricLabels)

  fig1.add_trace(go.Bar(x = list(summary_model.columns), y = list(summary_model.loc['MCC'])), 1, 1)
  fig1.add_trace(go.Bar(x = list(summary_model.columns), y = list(summary_model.loc['F1-Score'])), 1, 2)
  fig1.add_trace(go.Bar(x = list(summary_model.columns), y = list(summary_model.loc['F2-Score'])), 1, 3)
  fig1.add_trace(go.Bar(x = list(summary_model.columns), y = list(summary_model.loc['Recall'])), 1, 4)
  fig1.add_trace(go.Bar(x = list(summary_model.columns), y = list(summary_model.loc['Precision'])), 2, 1)
  fig1.add_trace(go.Bar(x = list(summary_model.columns), y = list(summary_model.loc['FM index'])), 2, 2)
  fig1.add_trace(go.Bar(x = list(summary_model.columns), y = list(summary_model.loc['Specificity'])), 2, 3)
  fig1.add_trace(go.Bar(x = list(summary_model.columns), y = list(summary_model.loc['Accuracy'])), 2, 4)

  fig1.update_layout(height = 600, width = 1000, coloraxis = dict(colorscale = 'Bluered_r'), showlegend = False)
  fig1.show()

In [ ]:
# Visual comparison of the model applied on different training sets through evaluation metrics

summary_visual(summary_logreg)

<a name='6.-KNN'></a>
# 6. $k$-Nearest Neighbors ($k$-NN)

In [ ]:
k = 29
knn = KNeighborsClassifier(n_neighbors = k, n_jobs = -1)

## Unaltered training set

In [ ]:
# Elements of confusion matrix

classification(knn, X_train_scaled_minmax, y_train, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_knn_unaltered = summary
summary_knn_unaltered.set_index('Metric')

y_pred_proba = knn.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_knn_unaltered_extended = summary.copy()
summary_knn_unaltered_extended.loc[len(summary_knn_unaltered_extended.index)] = ['ROC-AUC', roc_auc]
summary_knn_unaltered_extended.set_index('Metric')

summary_knn_unaltered_index = summary_knn_unaltered_extended.T
summary_knn_unaltered_index.columns = summary_knn_unaltered_index.iloc[0]
summary_knn_unaltered_index.drop(summary_knn_unaltered_index.index[0], inplace = True)
summary_knn_unaltered_index

## Random under-sampling

In [ ]:
# Elements of confusion matrix

classification(knn, X_train_under_scaled_minmax, y_train_under, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_knn_under = summary
summary_knn_under.set_index('Metric')

y_pred_proba = knn.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_knn_under_extended = summary.copy()
summary_knn_under_extended.loc[len(summary_knn_under_extended.index)] = ['ROC-AUC', roc_auc]
summary_knn_under_extended.set_index('Metric')

summary_knn_under_index = summary_knn_under_extended.T
summary_knn_under_index.columns = summary_knn_under_index.iloc[0]
summary_knn_under_index.drop(summary_knn_under_index.index[0], inplace = True)
summary_knn_under_index

## Random over-sampling

In [ ]:
# Elements of confusion matrix

classification(knn, X_train_over_scaled_minmax, y_train_over, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_knn_over = summary
summary_knn_over.set_index('Metric')

y_pred_proba = knn.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_knn_over_extended = summary.copy()
summary_knn_over_extended.loc[len(summary_knn_over_extended.index)] = ['ROC-AUC', roc_auc]
summary_knn_over_extended.set_index('Metric')

summary_knn_over_index = summary_knn_over_extended.T
summary_knn_over_index.columns = summary_knn_over_index.iloc[0]
summary_knn_over_index.drop(summary_knn_over_index.index[0], inplace = True)
summary_knn_over_index

## Random under-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(knn, X_train_under_imblearn_scaled_minmax, y_train_under_imblearn, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_knn_under_imblearn = summary
summary_knn_under_imblearn.set_index('Metric')

y_pred_proba = knn.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_knn_under_imblearn_extended = summary.copy()
summary_knn_under_imblearn_extended.loc[len(summary_knn_under_imblearn_extended.index)] = ['ROC-AUC', roc_auc]
summary_knn_under_imblearn_extended.set_index('Metric')

summary_knn_under_imblearn_index = summary_knn_under_imblearn_extended.T
summary_knn_under_imblearn_index.columns = summary_knn_under_imblearn_index.iloc[0]
summary_knn_under_imblearn_index.drop(summary_knn_under_imblearn_index.index[0], inplace = True)
summary_knn_under_imblearn_index

## Random over-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(knn, X_train_over_imblearn_scaled_minmax, y_train_over_imblearn, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_knn_over_imblearn = summary
summary_knn_over_imblearn.set_index('Metric')

y_pred_proba = knn.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_knn_over_imblearn_extended = summary.copy()
summary_knn_over_imblearn_extended.loc[len(summary_knn_over_imblearn_extended.index)] = ['ROC-AUC', roc_auc]
summary_knn_over_imblearn_extended.set_index('Metric')

summary_knn_over_imblearn_index = summary_knn_over_imblearn_extended.T
summary_knn_over_imblearn_index.columns = summary_knn_over_imblearn_index.iloc[0]
summary_knn_over_imblearn_index.drop(summary_knn_over_imblearn_index.index[0], inplace = True)
summary_knn_over_imblearn_index

## Synthetic minority over-sampling technique (SMOTE)

In [ ]:
# Elements of confusion matrix

classification(knn, X_train_over_smote_scaled_minmax, y_train_over_smote, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_knn_over_smote = summary
summary_knn_over_smote.set_index('Metric')

y_pred_proba = knn.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_knn_over_smote_extended = summary.copy()
summary_knn_over_smote_extended.loc[len(summary_knn_over_smote_extended.index)] = ['ROC-AUC', roc_auc]
summary_knn_over_smote_extended.set_index('Metric')

summary_knn_over_smote_index = summary_knn_over_smote_extended.T
summary_knn_over_smote_index.columns = summary_knn_over_smote_index.iloc[0]
summary_knn_over_smote_index.drop(summary_knn_over_smote_index.index[0], inplace = True)
summary_knn_over_smote_index

## Under-sampling via NearMiss

In [ ]:
# Elements of confusion matrix

classification(knn, X_train_under_nm_scaled_minmax, y_train_under_nm, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_knn_under_nm = summary
summary_knn_under_nm.set_index('Metric')

y_pred_proba = knn.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_knn_under_nm_extended = summary.copy()
summary_knn_under_nm_extended.loc[len(summary_knn_under_nm_extended.index)] = ['ROC-AUC', roc_auc]
summary_knn_under_nm_extended.set_index('Metric')

summary_knn_under_nm_index = summary_knn_under_nm_extended.T
summary_knn_under_nm_index.columns = summary_knn_under_nm_index.iloc[0]
summary_knn_under_nm_index.drop(summary_knn_under_nm_index.index[0], inplace = True)
summary_knn_under_nm_index

## Summary of $k$-NN classification models

In [ ]:
summary_knn = pd.DataFrame(columns = ['Metric'])

summary_knn['Metric'] = EvalMetricLabels
summary_knn_list = [summary_knn_unaltered, summary_knn_under, summary_knn_over, summary_knn_under_imblearn,
                    summary_knn_over_imblearn, summary_knn_over_smote, summary_knn_under_nm]

for i in summary_knn_list:
    summary_knn = pd.merge(summary_knn, i, on = 'Metric')
    
TrainingSetsMetric = TrainingSets.copy()
TrainingSetsMetric.insert(0, 'Metric')

summary_knn.columns = TrainingSetsMetric
summary_knn.set_index('Metric', inplace = True)
summary_knn

In [ ]:
# Visual comparison of the model applied on different training sets through various evaluation metrics

summary_visual(summary_knn)

**Note:** A potential issue with $k$-NN classification models, which is relevant in this project is that, they are affected by **curse of dimensionality**, as well as **presence of outliers in the feature variables**. Despite that, it performs fairly well when applied on the unaltered (imbalanced) training set, in particular with respect to **MCC**, but **F2-score** as well.

<a name='7.-Decision-Tree'></a>
# 7. Decision Tree

In [ ]:
dt = DecisionTreeClassifier()

## Unaltered training set

In [ ]:
# Elements of confusion matrix

classification(dt, X_train, y_train, X_test, y_test)

# Summary of evaluation metrics

summary_dt_unaltered = summary
summary_dt_unaltered.set_index('Metric')

y_pred_proba = dt.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_dt_unaltered_extended = summary.copy()
summary_dt_unaltered_extended.loc[len(summary_dt_unaltered_extended.index)] = ['ROC-AUC', roc_auc]
summary_dt_unaltered_extended.set_index('Metric')

summary_dt_unaltered_index = summary_dt_unaltered_extended.T
summary_dt_unaltered_index.columns = summary_dt_unaltered_index.iloc[0]
summary_dt_unaltered_index.drop(summary_dt_unaltered_index.index[0], inplace = True)
summary_dt_unaltered_index

## Random under-sampling

In [ ]:
# Elements of confusion matrix

classification(dt, X_train_under, y_train_under, X_test, y_test)

# Summary of evaluation metrics

summary_dt_under = summary
summary_dt_under.set_index('Metric')

y_pred_proba = dt.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_dt_under_extended = summary.copy()
summary_dt_under_extended.loc[len(summary_dt_under_extended.index)] = ['ROC-AUC', roc_auc]
summary_dt_under_extended.set_index('Metric')

summary_dt_under_index = summary_dt_under_extended.T
summary_dt_under_index.columns = summary_dt_under_index.iloc[0]
summary_dt_under_index.drop(summary_dt_under_index.index[0], inplace = True)
summary_dt_under_index

## Random over-sampling

In [ ]:
# Elements of confusion matrix

classification(dt, X_train_over, y_train_over, X_test, y_test)

# Summary of evaluation metrics

summary_dt_over = summary
summary_dt_over.set_index('Metric')

y_pred_proba = dt.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_dt_over_extended = summary.copy()
summary_dt_over_extended.loc[len(summary_dt_over_extended.index)] = ['ROC-AUC', roc_auc]
summary_dt_over_extended.set_index('Metric')

summary_dt_over_index = summary_dt_over_extended.T
summary_dt_over_index.columns = summary_dt_over_index.iloc[0]
summary_dt_over_index.drop(summary_dt_over_index.index[0], inplace = True)
summary_dt_over_index

## Random under-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(dt, X_train_under_imblearn, y_train_under_imblearn, X_test, y_test)

# Summary of evaluation metrics

summary_dt_under_imblearn = summary
summary_dt_under_imblearn.set_index('Metric')

y_pred_proba = dt.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_dt_under_imblearn_extended = summary.copy()
summary_dt_under_imblearn_extended.loc[len(summary_dt_under_imblearn_extended.index)] = ['ROC-AUC', roc_auc]
summary_dt_under_imblearn_extended.set_index('Metric')

summary_dt_under_imblearn_index = summary_dt_under_imblearn_extended.T
summary_dt_under_imblearn_index.columns = summary_dt_under_imblearn_index.iloc[0]
summary_dt_under_imblearn_index.drop(summary_dt_under_imblearn_index.index[0], inplace = True)
summary_dt_under_imblearn_index

## Random over-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(dt, X_train_over_imblearn, y_train_over_imblearn, X_test, y_test)

# Summary of evaluation metrics

summary_dt_over_imblearn = summary
summary_dt_over_imblearn.set_index('Metric')

y_pred_proba = dt.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_dt_over_imblearn_extended = summary.copy()
summary_dt_over_imblearn_extended.loc[len(summary_dt_over_imblearn_extended.index)] = ['ROC-AUC', roc_auc]
summary_dt_over_imblearn_extended.set_index('Metric')

summary_dt_over_imblearn_index = summary_dt_over_imblearn_extended.T
summary_dt_over_imblearn_index.columns = summary_dt_over_imblearn_index.iloc[0]
summary_dt_over_imblearn_index.drop(summary_dt_over_imblearn_index.index[0], inplace = True)
summary_dt_over_imblearn_index

## Synthetic minority over-sampling technique (SMOTE)

In [ ]:
# Elements of confusion matrix

classification(dt, X_train_over_smote, y_train_over_smote, X_test, y_test)

# Summary of evaluation metrics

summary_dt_over_smote = summary
summary_dt_over_smote.set_index('Metric')

y_pred_proba = dt.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_dt_over_smote_extended = summary.copy()
summary_dt_over_smote_extended.loc[len(summary_dt_over_smote_extended.index)] = ['ROC-AUC', roc_auc]
summary_dt_over_smote_extended.set_index('Metric')

summary_dt_over_smote_index = summary_dt_over_smote_extended.T
summary_dt_over_smote_index.columns = summary_dt_over_smote_index.iloc[0]
summary_dt_over_smote_index.drop(summary_dt_over_smote_index.index[0], inplace = True)
summary_dt_over_smote_index

## Under-sampling via NearMiss

In [ ]:
# Elements of confusion matrix

classification(dt, X_train_under_nm, y_train_under_nm, X_test, y_test)

# Summary of evaluation metrics

summary_dt_under_nm = summary
summary_dt_under_nm.set_index('Metric')

y_pred_proba = dt.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_dt_under_nm_extended = summary.copy()
summary_dt_under_nm_extended.loc[len(summary_dt_under_nm_extended.index)] = ['ROC-AUC', roc_auc]
summary_dt_under_nm_extended.set_index('Metric')

summary_dt_under_nm_index = summary_dt_under_nm_extended.T
summary_dt_under_nm_index.columns = summary_dt_under_nm_index.iloc[0]
summary_dt_under_nm_index.drop(summary_dt_under_nm_index.index[0])
summary_dt_under_nm_index

## Summary of decision tree classification models

In [ ]:
summary_dt = pd.DataFrame(columns = ['Metric'])

EvalMetricLabels_dt = EvalMetricLabels
summary_dt['Metric'] = EvalMetricLabels
summary_dt_list = [summary_dt_unaltered, summary_dt_under, summary_dt_over, summary_dt_under_imblearn,
                    summary_dt_over_imblearn, summary_dt_over_smote, summary_dt_under_nm]

for i in summary_dt_list:
    summary_dt = pd.merge(summary_dt, i, on = 'Metric')
    
TrainingSetsMetric = TrainingSets.copy()
TrainingSetsMetric.insert(0, 'Metric')

summary_dt.columns = TrainingSetsMetric
summary_dt.set_index('Metric', inplace = True)
summary_dt

In [ ]:
# Visual comparison of the model applied on different training sets through various evaluation metrics

summary_visual(summary_dt)

<a name='8.-SVM'></a>
# 8. Support Vector Machine (SVM)

In [ ]:
svm_linear = svm.SVC(kernel = 'linear')

## Unaltered training set

In [ ]:
# Elements of confusion matrix

classification(svm_linear, X_train_scaled_minmax, y_train, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_svm_linear_unaltered = summary
summary_svm_linear_unaltered.set_index('Metric')

summary_svm_linear_unaltered_index = summary_svm_linear_unaltered.T
summary_svm_linear_unaltered_index.columns = summary_svm_linear_unaltered_index.iloc[0]
summary_svm_linear_unaltered_index.drop(summary_svm_linear_unaltered_index.index[0], inplace = True)
summary_svm_linear_unaltered_index

# classification(svm_linear, X_train, y_train, X_test, y_test) # TP = 37, FN = 75, TN = 56840, FP = 10

## Random under-sampling

In [ ]:
# Elements of confusion matrix

classification(svm_linear, X_train_under_scaled_minmax, y_train_under, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_svm_linear_under = summary
summary_svm_linear_under.set_index('Metric')

summary_svm_linear_under_index = summary_svm_linear_under.T
summary_svm_linear_under_index.columns = summary_svm_linear_under_index.iloc[0]
summary_svm_linear_under_index.drop(summary_svm_linear_under_index.index[0], inplace = True)
summary_svm_linear_under_index

## Random over-sampling

In [ ]:
# Elements of confusion matrix

classification(svm_linear, X_train_over_scaled_minmax, y_train_over, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_svm_linear_over = summary
summary_svm_linear_over.set_index('Metric')

summary_svm_linear_over_index = summary_svm_linear_over.T
summary_svm_linear_over_index.columns = summary_svm_linear_over_index.iloc[0]
summary_svm_linear_over_index.drop(summary_svm_linear_over_index.index[0], inplace = True)
summary_svm_linear_over_index

## Random under-sampling with imbalanced-learning library

In [ ]:
# Elements of confusion matrix

classification(svm_linear, X_train_under_imblearn_scaled_minmax, y_train_under_imblearn, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_svm_linear_under_imblearn = summary
summary_svm_linear_under_imblearn.set_index('Metric')

summary_svm_linear_under_imblearn_index = summary_svm_linear_under_imblearn.T
summary_svm_linear_under_imblearn_index.columns = summary_svm_linear_under_imblearn_index.iloc[0]
summary_svm_linear_under_imblearn_index.drop(summary_svm_linear_under_imblearn_index.index[0], inplace = True)
summary_svm_linear_under_imblearn_index

## Random over-sampling with imbalanced-learning library

In [ ]:
# Elements of confusion matrix

classification(svm_linear, X_train_over_imblearn_scaled_minmax, y_train_over_imblearn, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_svm_linear_over_imblearn = summary
summary_svm_linear_over_imblearn.set_index('Metric')

summary_svm_linear_over_imblearn_index = summary_svm_linear_over_imblearn.T
summary_svm_linear_over_imblearn_index.columns = summary_svm_linear_over_imblearn_index.iloc[0]
summary_svm_linear_over_imblearn_index.drop(summary_svm_linear_over_imblearn_index.index[0], inplace = True)
summary_svm_linear_over_imblearn_index

## Synthetic minority over-sampling technique (SMOTE)

In [ ]:
# Elements of confusion matrix

classification(svm_linear, X_train_over_smote_scaled_minmax, y_train_over_smote, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_svm_linear_over_smote = summary
summary_svm_linear_over_smote.set_index('Metric')

summary_svm_linear_over_smote_index = summary_svm_linear_over_smote.T
summary_svm_linear_over_smote_index.columns = summary_svm_linear_over_smote_index.iloc[0]
summary_svm_linear_over_smote_index.drop(summary_svm_linear_over_smote_index.index[0], inplace = True)
summary_svm_linear_over_smote_index

## Under-sampling via NearMiss

In [ ]:
# Elements of confusion matrix

classification(svm_linear, X_train_under_nm_scaled_minmax, y_train_under_nm, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_svm_linear_under_nm = summary
summary_svm_linear_under_nm.set_index('Metric')

summary_svm_linear_under_nm_index = summary_svm_linear_under_nm.T
summary_svm_linear_under_nm_index.columns = summary_svm_linear_under_nm_index.iloc[0]
summary_svm_linear_under_nm_index.drop(summary_svm_linear_under_nm_index.index[0], inplace = True)
summary_svm_linear_under_nm_index

## Summary of linear SVM classification models

In [ ]:
summary_svm_linear = pd.DataFrame(columns = ['Metric'])

summary_svm_linear['Metric'] = EvalMetricLabels
summary_svm_linear_list = [summary_svm_linear_unaltered, summary_svm_linear_under, summary_svm_linear_over,
                           summary_svm_linear_under_imblearn, summary_svm_linear_over_imblearn,
                           summary_svm_linear_over_smote, summary_svm_linear_under_nm]

for i in summary_svm_linear_list:
    summary_svm_linear = pd.merge(summary_svm_linear, i, on = 'Metric')

TrainingSetsMetric = TrainingSets.copy()
TrainingSetsMetric.insert(0, 'Metric')

summary_svm_linear.columns = TrainingSetsMetric
summary_svm_linear.set_index('Metric', inplace = True)
summary_svm_linear

In [ ]:
# Visual comparison of the model applied on different training sets through various evaluation metrics

summary_visual(summary_svm_linear)

fig1 = make_subplots(rows = 4, cols = 2, shared_yaxes = True, subplot_titles = EvalMetricLabels)

fig1.add_trace(go.Bar(x = list(summary_svm_linear.columns), y = list(summary_svm_linear.loc['MCC'])), 1, 1)
fig1.add_trace(go.Bar(x = list(summary_svm_linear.columns), y = list(summary_svm_linear.loc['F1-Score'])), 1, 2)
fig1.add_trace(go.Bar(x = list(summary_svm_linear.columns), y = list(summary_svm_linear.loc['F2-Score'])), 2, 1)
fig1.add_trace(go.Bar(x = list(summary_svm_linear.columns), y = list(summary_svm_linear.loc['Recall'])), 2, 2)
fig1.add_trace(go.Bar(x = list(summary_svm_linear.columns), y = list(summary_svm_linear.loc['Precision'])), 3, 1)
fig1.add_trace(go.Bar(x = list(summary_svm_linear.columns), y = list(summary_svm_linear.loc['FM index'])), 3, 2)
fig1.add_trace(go.Bar(x = list(summary_svm_linear.columns), y = list(summary_svm_linear.loc['Accuracy'])), 4, 1)
fig1.add_trace(go.Bar(x = list(summary_svm_linear.columns), y = list(summary_svm_linear.loc['Specificity'])), 4, 2)

fig1.update_layout(height = 2000, width = 800, coloraxis = dict(colorscale='Bluered_r'), showlegend = False)
fig1.show()

<a name='9.-Naive-Bayes'></a>
# 9. Naive Bayes

In [ ]:
nb = GaussianNB()

## Unaltered training set

In [ ]:
# Elements of confusion matrix

classification(nb, X_train, y_train, X_test, y_test)

# Summary of evaluation metrics

summary_nb_unaltered = summary.copy()
summary_nb_unaltered.set_index('Metric')

y_pred_proba = nb.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_nb_unaltered_extended = summary.copy()
summary_nb_unaltered_extended.loc[len(summary_nb_unaltered_extended.index)] = ['ROC-AUC', roc_auc]
summary_nb_unaltered_extended.set_index('Metric')

summary_nb_unaltered_index = summary_nb_unaltered_extended.T
summary_nb_unaltered_index.columns = summary_nb_unaltered_index.iloc[0]
summary_nb_unaltered_index.drop(summary_nb_unaltered_index.index[0], inplace = True)
summary_nb_unaltered_index

## Random under-sampling

In [ ]:
# Elements of confusion matrix

classification(nb, X_train_under, y_train_under, X_test, y_test)

# Summary of evaluation metrics

summary_nb_under = summary.copy()
summary_nb_under.set_index('Metric')

y_pred_proba = nb.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_nb_under_extended = summary.copy()
summary_nb_under_extended.loc[len(summary_nb_under_extended.index)] = ['ROC-AUC', roc_auc]
summary_nb_under_extended.set_index('Metric')

summary_nb_under_index = summary_nb_under_extended.T
summary_nb_under_index.columns = summary_nb_under_index.iloc[0]
summary_nb_under_index.drop(summary_nb_under_index.index[0], inplace = True)
summary_nb_under_index

## Random over-sampling

In [ ]:
# Elements of confusion matrix

classification(nb, X_train_over, y_train_over, X_test, y_test)

# Summary of evaluation metrics

summary_nb_over = summary.copy()
summary_nb_over.set_index('Metric')

y_pred_proba = nb.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_nb_over_extended = summary.copy()
summary_nb_over_extended.loc[len(summary_nb_over_extended.index)] = ['ROC-AUC', roc_auc]
summary_nb_over_extended.set_index('Metric')

summary_nb_over_index = summary_nb_over_extended.T
summary_nb_over_index.columns = summary_nb_over_index.iloc[0]
summary_nb_over_index.drop(summary_nb_over_index.index[0], inplace = True)
summary_nb_over_index

## Random under-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(nb, X_train_under_imblearn, y_train_under_imblearn, X_test, y_test)

# Summary of evaluation metrics

summary_nb_under_imblearn = summary.copy()
summary_nb_under_imblearn.set_index('Metric')

y_pred_proba = nb.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_nb_under_imblearn_extended = summary.copy()
summary_nb_under_imblearn_extended.loc[len(summary_nb_under_imblearn_extended.index)] = ['ROC-AUC', roc_auc]
summary_nb_under_imblearn_extended.set_index('Metric')

summary_nb_under_imblearn_index = summary_nb_under_imblearn_extended.T
summary_nb_under_imblearn_index.columns = summary_nb_under_imblearn_index.iloc[0]
summary_nb_under_imblearn_index.drop(summary_nb_under_imblearn_index.index[0], inplace = True)
summary_nb_under_imblearn_index

## Random over-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(nb, X_train_over_imblearn, y_train_over_imblearn, X_test, y_test)

# Summary of evaluation metrics

summary_nb_over_imblearn = summary.copy()
summary_nb_over_imblearn.set_index('Metric')

y_pred_proba = nb.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_nb_over_imblearn_extended = summary.copy()
summary_nb_over_imblearn_extended.loc[len(summary_nb_over_imblearn_extended.index)] = ['ROC-AUC', roc_auc]
summary_nb_over_imblearn_extended.set_index('Metric')

summary_nb_over_imblearn_index = summary_nb_over_imblearn_extended.T
summary_nb_over_imblearn_index.columns = summary_nb_over_imblearn_index.iloc[0]
summary_nb_over_imblearn_index.drop(summary_nb_over_imblearn_index.index[0], inplace = True)
summary_nb_over_imblearn_index

## Synthetic minority over-sampling technique (SMOTE)

In [ ]:
# Elements of confusion matrix

classification(nb, X_train_over_smote, y_train_over_smote, X_test, y_test)

# Summary of evaluation metrics

summary_nb_over_smote = summary.copy()
summary_nb_over_smote.set_index('Metric')

y_pred_proba = nb.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_nb_over_smote_extended = summary.copy()
summary_nb_over_smote_extended.loc[len(summary_nb_over_smote_extended.index)] = ['ROC-AUC', roc_auc]
summary_nb_over_smote_extended.set_index('Metric')

summary_nb_over_smote_index = summary_nb_over_smote_extended.T
summary_nb_over_smote_index.columns = summary_nb_over_smote_index.iloc[0]
summary_nb_over_smote_index.drop(summary_nb_over_smote_index.index[0], inplace = True)
summary_nb_over_smote_index

## Under-sampling via NearMiss

In [ ]:
# Elements of confusion matrix

classification(nb, X_train_under_nm, y_train_under_nm, X_test, y_test)

# Summary of evaluation metrics

summary_nb_under_nm = summary.copy()
summary_nb_under_nm.set_index('Metric')

y_pred_proba = nb.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_nb_under_nm_extended = summary.copy()
summary_nb_under_nm_extended.loc[len(summary_nb_under_nm_extended.index)] = ['ROC-AUC', roc_auc]
summary_nb_under_nm_extended.set_index('Metric')

summary_nb_under_nm_index = summary_nb_under_nm_extended.T
summary_nb_under_nm_index.columns = summary_nb_under_nm_index.iloc[0]
summary_nb_under_nm_index.drop(summary_nb_under_nm_index.index[0], inplace = True)
summary_nb_under_nm_index

## Summary of naive Bayes models

In [ ]:
summary_nb = pd.DataFrame(columns = ['Metric'])

summary_nb['Metric'] = EvalMetricLabels
summary_nb_list = [summary_nb_unaltered, summary_nb_under, summary_nb_over, summary_nb_under_imblearn,
                       summary_nb_over_imblearn, summary_nb_over_smote, summary_nb_under_nm]

for i in summary_nb_list:
    summary_nb = pd.merge(summary_nb, i, on = 'Metric')
    
TrainingSetsMetric = TrainingSets.copy()
TrainingSetsMetric.insert(0, 'Metric')

summary_nb.columns = TrainingSetsMetric
summary_nb.set_index('Metric', inplace = True)
summary_nb

In [ ]:
# Visual comparison of the model applied on different training sets through various evaluation metrics

summary_visual(summary_nb)

<a name='10.-Random-Forest'></a>
# 10. Random Forest

The Random Forest classifier employs multiple decision trees, thereby avoiding the reliance upon feature selection of a singular decision tree.

In [ ]:
rf = RandomForestClassifier(n_estimators = 100)

## Unaltered training set

In [ ]:
# Elements of confusion matrix

classification(rf, X_train, y_train, X_test, y_test)

# Summary of evaluation metrics

summary_rf_unaltered = summary.copy()
summary_rf_unaltered.set_index('Metric')

y_pred_proba = rf.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_rf_unaltered_extended = summary.copy()
summary_rf_unaltered_extended.loc[len(summary_rf_unaltered_extended.index)] = ['ROC-AUC', roc_auc]
summary_rf_unaltered_extended.set_index('Metric')

summary_rf_unaltered_index = summary_rf_unaltered_extended.T
summary_rf_unaltered_index.columns = summary_rf_unaltered_index.iloc[0]
summary_rf_unaltered_index.drop(summary_rf_unaltered_index.index[0], inplace = True)
summary_rf_unaltered_index

## Random under-sampling

In [ ]:
# Elements of confusion matrix

classification(rf, X_train_under, y_train_under, X_test, y_test)

# Summary of evaluation metrics

summary_rf_under = summary.copy()
summary_rf_under.set_index('Metric')

y_pred_proba = rf.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_rf_under_extended = summary.copy()
summary_rf_under_extended.loc[len(summary_rf_under_extended.index)] = ['ROC-AUC', roc_auc]
summary_rf_under_extended.set_index('Metric')

summary_rf_under_index = summary_rf_under_extended.T
summary_rf_under_index.columns = summary_rf_under_index.iloc[0]
summary_rf_under_index.drop(summary_rf_under_index.index[0], inplace = True)
summary_rf_under_index

## Random over-sampling

In [ ]:
# Elements of confusion matrix

classification(rf, X_train_over, y_train_over, X_test, y_test)

# Summary of evaluation metrics

summary_rf_over = summary.copy()
summary_rf_over.set_index('Metric')

y_pred_proba = rf.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_rf_over_extended = summary.copy()
summary_rf_over_extended.loc[len(summary_rf_over_extended.index)] = ['ROC-AUC', roc_auc]
summary_rf_over_extended.set_index('Metric')

summary_rf_over_index = summary_rf_over_extended.T
summary_rf_over_index.columns = summary_rf_over_index.iloc[0]
summary_rf_over_index.drop(summary_rf_over_index.index[0], inplace = True)
summary_rf_over_index

## Random under-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(rf, X_train_under_imblearn, y_train_under_imblearn, X_test, y_test)

# Summary of evaluation metrics

summary_rf_under_imblearn = summary.copy()
summary_rf_under_imblearn.set_index('Metric')

y_pred_proba = rf.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_rf_under_imblearn_extended = summary.copy()
summary_rf_under_imblearn_extended.loc[len(summary_rf_under_imblearn_extended.index)] = ['ROC-AUC', roc_auc]
summary_rf_under_imblearn_extended.set_index('Metric')

summary_rf_under_imblearn_index = summary_rf_under_imblearn_extended.T
summary_rf_under_imblearn_index.columns = summary_rf_under_imblearn_index.iloc[0]
summary_rf_under_imblearn_index.drop(summary_rf_under_imblearn_index.index[0], inplace = True)
summary_rf_under_imblearn_index

## Random over-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(rf, X_train_over_imblearn, y_train_over_imblearn, X_test, y_test)

# Summary of evaluation metrics

summary_rf_over_imblearn = summary.copy()
summary_rf_over_imblearn.set_index('Metric')

y_pred_proba = rf.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_rf_over_imblearn_extended = summary.copy()
summary_rf_over_imblearn_extended.loc[len(summary_rf_over_imblearn_extended.index)] = ['ROC-AUC', roc_auc]
summary_rf_over_imblearn_extended.set_index('Metric')

summary_rf_over_imblearn_index = summary_rf_over_imblearn_extended.T
summary_rf_over_imblearn_index.columns = summary_rf_over_imblearn_index.iloc[0]
summary_rf_over_imblearn_index.drop(summary_rf_over_imblearn_index.index[0], inplace = True)
summary_rf_over_imblearn_index

## Synthetic minority over-sampling technique (SMOTE)

In [ ]:
# Elements of confusion matrix

classification(rf, X_train_over_smote, y_train_over_smote, X_test, y_test)

# Summary of evaluation metrics

summary_rf_over_smote = summary.copy()
summary_rf_over_smote.set_index('Metric')

y_pred_proba = rf.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_rf_over_smote_extended = summary.copy()
summary_rf_over_smote_extended.loc[len(summary_rf_over_smote_extended.index)] = ['ROC-AUC', roc_auc]
summary_rf_over_smote_extended.set_index('Metric')

summary_rf_over_smote_index = summary_rf_over_smote_extended.T
summary_rf_over_smote_index.columns = summary_rf_over_smote_index.iloc[0]
summary_rf_over_smote_index.drop(summary_rf_over_smote_index.index[0], inplace = True)
summary_rf_over_smote_index

## Under-sampling via NearMiss

In [ ]:
# Elements of confusion matrix

classification(rf, X_train_under_nm, y_train_under_nm, X_test, y_test)

# Summary of evaluation metrics

summary_rf_under_nm = summary.copy()
summary_rf_under_nm.set_index('Metric')

y_pred_proba = rf.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_rf_under_nm_extended = summary.copy()
summary_rf_under_nm_extended.loc[len(summary_rf_under_nm_extended.index)] = ['ROC-AUC', roc_auc]
summary_rf_under_nm_extended.set_index('Metric')

summary_rf_under_nm_index = summary_rf_under_nm_extended.T
summary_rf_under_nm_index.columns = summary_rf_under_nm_index.iloc[0]
summary_rf_under_nm_index.drop(summary_rf_under_nm_index.index[0], inplace = True)
summary_rf_under_nm_index

## Summary of random forest models

In [ ]:
summary_rf = pd.DataFrame(columns = ['Metric'])

summary_rf['Metric'] = EvalMetricLabels
summary_rf_list = [summary_rf_unaltered, summary_rf_under, summary_rf_over, summary_rf_under_imblearn,
                   summary_rf_over_imblearn, summary_rf_over_smote, summary_rf_under_nm]

for i in summary_rf_list:
    summary_rf = pd.merge(summary_rf, i, on = 'Metric')
    
TrainingSetsMetric = TrainingSets.copy()
TrainingSetsMetric.insert(0, 'Metric')

summary_rf.columns = TrainingSetsMetric
summary_rf.set_index('Metric', inplace = True)
summary_rf

In [ ]:
# Visual comparison of the model applied on different training sets through various evaluation metrics

summary_visual(summary_rf)

<a name='11.-LDA'></a>
# 11. Linear discriminant analysis (LDA)

In [ ]:
lda = LinearDiscriminantAnalysis()

## Unaltered training set

In [ ]:
# Elements of confusion matrix

classification(lda, X_train, y_train, X_test, y_test)

# Summary of evaluation metrics

summary_lda_unaltered = summary.copy()
summary_lda_unaltered.set_index('Metric')

y_score = lda.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = lda.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_lda_unaltered_extended = summary.copy()
summary_lda_unaltered_extended.loc[len(summary_lda_unaltered_extended.index)] = ['AP', average_precision]
summary_lda_unaltered_extended.loc[len(summary_lda_unaltered_extended.index)] = ['ROC-AUC', roc_auc]
summary_lda_unaltered_extended.set_index('Metric')

summary_lda_unaltered_index = summary_lda_unaltered_extended.T
summary_lda_unaltered_index.columns = summary_lda_unaltered_index.iloc[0]
summary_lda_unaltered_index.drop(summary_lda_unaltered_index.index[0], inplace = True)
summary_lda_unaltered_index

## Random under-sampling

In [ ]:
# Elements of confusion matrix

classification(lda, X_train_under, y_train_under, X_test, y_test)

# Summary of evaluation metrics

summary_lda_under = summary
summary_lda_under.set_index('Metric')

y_score = lda.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = lda.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_lda_under_extended = summary.copy()
summary_lda_under_extended.loc[len(summary_lda_under_extended.index)] = ['AP', average_precision]
summary_lda_under_extended.loc[len(summary_lda_under_extended.index)] = ['ROC-AUC', roc_auc]
summary_lda_under_extended.set_index('Metric')

summary_lda_under_index = summary_lda_under_extended.T
summary_lda_under_index.columns = summary_lda_under_index.iloc[0]
summary_lda_under_index.drop(summary_lda_under_index.index[0], inplace = True)
summary_lda_under_index

## Random over-sampling

In [ ]:
# Elements of confusion matrix

classification(lda, X_train_over, y_train_over, X_test, y_test)

# Summary of evaluation metrics

summary_lda_over = summary
summary_lda_over.set_index('Metric')

y_score = lda.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = lda.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_lda_over_extended = summary.copy()
summary_lda_over_extended.loc[len(summary_lda_over_extended.index)] = ['AP', average_precision]
summary_lda_over_extended.loc[len(summary_lda_over_extended.index)] = ['ROC-AUC', roc_auc]
summary_lda_over_extended.set_index('Metric')

summary_lda_over_index = summary_lda_over_extended.T
summary_lda_over_index.columns = summary_lda_over_index.iloc[0]
summary_lda_over_index.drop(summary_lda_over_index.index[0], inplace = True)
summary_lda_over_index

## Random under-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(lda, X_train_under_imblearn, y_train_under_imblearn, X_test, y_test)

# Summary of evaluation metrics

summary_lda_under_imblearn = summary
summary_lda_under_imblearn.set_index('Metric')

y_score = lda.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = lda.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_lda_under_imblearn_extended = summary.copy()
summary_lda_under_imblearn_extended.loc[len(summary_lda_under_imblearn_extended.index)] = ['AP', average_precision]
summary_lda_under_imblearn_extended.loc[len(summary_lda_under_imblearn_extended.index)] = ['ROC-AUC', roc_auc]
summary_lda_under_imblearn_extended.set_index('Metric')

summary_lda_under_imblearn_index = summary_lda_under_imblearn_extended.T
summary_lda_under_imblearn_index.columns = summary_lda_under_imblearn_index.iloc[0]
summary_lda_under_imblearn_index.drop(summary_lda_under_imblearn_index.index[0], inplace = True)
summary_lda_under_imblearn_index

## Random over-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(lda, X_train_over_imblearn, y_train_over_imblearn, X_test, y_test)

# Summary of evaluation metrics

summary_lda_over_imblearn = summary
summary_lda_over_imblearn.set_index('Metric')

y_score = lda.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = lda.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_lda_over_imblearn_extended = summary.copy()
summary_lda_over_imblearn_extended.loc[len(summary_lda_over_imblearn_extended.index)] = ['AP', average_precision]
summary_lda_over_imblearn_extended.loc[len(summary_lda_over_imblearn_extended.index)] = ['ROC-AUC', roc_auc]
summary_lda_over_imblearn_extended.set_index('Metric')

summary_lda_over_imblearn_index = summary_lda_over_imblearn_extended.T
summary_lda_over_imblearn_index.columns = summary_lda_over_imblearn_index.iloc[0]
summary_lda_over_imblearn_index.drop(summary_lda_over_imblearn_index.index[0], inplace = True)
summary_lda_over_imblearn_index

## Synthetic minority over-sampling technique (SMOTE)

In [ ]:
# Elements of confusion matrix

classification(lda, X_train_over_smote, y_train_over_smote, X_test, y_test)

# Summary of evaluation metrics

summary_lda_over_smote = summary
summary_lda_over_smote.set_index('Metric')

y_score = lda.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = lda.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_lda_over_smote_extended = summary.copy()
summary_lda_over_smote_extended.loc[len(summary_lda_over_smote_extended.index)] = ['AP', average_precision]
summary_lda_over_smote_extended.loc[len(summary_lda_over_smote_extended.index)] = ['ROC-AUC', roc_auc]
summary_lda_over_smote_extended.set_index('Metric')

summary_lda_over_smote_index = summary_lda_over_smote_extended.T
summary_lda_over_smote_index.columns = summary_lda_over_smote_index.iloc[0]
summary_lda_over_smote_index.drop(summary_lda_over_smote_index.index[0], inplace = True)
summary_lda_over_smote_index

## Under-sampling via NearMiss

In [ ]:
# Elements of confusion matrix

classification(lda, X_train_under_nm, y_train_under_nm, X_test, y_test)

# Summary of evaluation metrics

summary_lda_under_nm = summary
summary_lda_under_nm.set_index('Metric')

y_score = lda.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)
y_pred_proba = lda.predict_proba(X_test)[::,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)

summary_lda_under_nm_extended = summary.copy()
summary_lda_under_nm_extended.loc[len(summary_lda_under_nm_extended.index)] = ['AP', average_precision]
summary_lda_under_nm_extended.loc[len(summary_lda_under_nm_extended.index)] = ['ROC-AUC', roc_auc]
summary_lda_under_nm_extended.set_index('Metric')

summary_lda_under_nm_index = summary_lda_under_nm_extended.T
summary_lda_under_nm_index.columns = summary_lda_under_nm_index.iloc[0]
summary_lda_under_nm_index.drop(summary_lda_under_nm_index.index[0], inplace = True)
summary_lda_under_nm_index

## Summary of LDA models

In [ ]:
summary_lda = pd.DataFrame(columns = ['Metric'])

summary_lda['Metric'] = EvalMetricLabels
summary_lda_list = [summary_lda_unaltered, summary_lda_under, summary_lda_over, summary_lda_under_imblearn,
                    summary_lda_over_imblearn, summary_lda_over_smote, summary_lda_under_nm]

for i in summary_lda_list:
    summary_lda = pd.merge(summary_lda, i, on = 'Metric')
    
TrainingSetsMetric = TrainingSets.copy()
TrainingSetsMetric.insert(0, 'Metric')

summary_lda.columns = TrainingSetsMetric
summary_lda.set_index('Metric', inplace = True)
summary_lda

In [ ]:
# Visual comparison of the model applied on different training sets through various evaluation metrics

summary_visual(summary_lda)

<a name='12.-SGD'></a>
# 12. Stochastic Gradient Descent (SGD)

In [ ]:
sgd = SGDClassifier(loss = 'hinge')

## Unaltered training set

In [ ]:
# Elements of confusion matrix

classification(sgd, X_train_scaled_minmax, y_train, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_sgd_unaltered = summary.copy()
summary_sgd_unaltered.set_index('Metric')

y_score = sgd.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_sgd_unaltered_extended = summary.copy()
summary_sgd_unaltered_extended.loc[len(summary_sgd_unaltered_extended.index)] = ['AP', average_precision]
summary_sgd_unaltered_extended.set_index('Metric')

summary_sgd_unaltered_index = summary_sgd_unaltered_extended.T
summary_sgd_unaltered_index.columns = summary_sgd_unaltered_index.iloc[0]
summary_sgd_unaltered_index.drop(summary_sgd_unaltered_index.index[0], inplace = True)
summary_sgd_unaltered_index

## Random under-sampling

In [ ]:
# Elements of confusion matrix

classification(sgd, X_train_under_scaled_minmax, y_train_under, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_sgd_under = summary
summary_sgd_under.set_index('Metric')

y_score = sgd.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_sgd_under_extended = summary.copy()
summary_sgd_under_extended.loc[len(summary_sgd_under_extended.index)] = ['AP', average_precision]
summary_sgd_under_extended.set_index('Metric')

summary_sgd_under_index = summary_sgd_under_extended.T
summary_sgd_under_index.columns = summary_sgd_under_index.iloc[0]
summary_sgd_under_index.drop(summary_sgd_under_index.index[0], inplace = True)
summary_sgd_under_index

## Random over-sampling

In [ ]:
# Elements of confusion matrix

classification(sgd, X_train_over_scaled_minmax, y_train_over, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_sgd_over = summary
summary_sgd_over.set_index('Metric')

y_score = sgd.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_sgd_over_extended = summary.copy()
summary_sgd_over_extended.loc[len(summary_sgd_over_extended.index)] = ['AP', average_precision]
summary_sgd_over_extended.set_index('Metric')

summary_sgd_over_index = summary_sgd_over_extended.T
summary_sgd_over_index.columns = summary_sgd_over_index.iloc[0]
summary_sgd_over_index.drop(summary_sgd_over_index.index[0], inplace = True)
summary_sgd_over_index

## Random under-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(sgd, X_train_under_imblearn_scaled_minmax, y_train_under_imblearn, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_sgd_under_imblearn = summary
summary_sgd_under_imblearn.set_index('Metric')

y_score = sgd.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_sgd_under_imblearn_extended = summary.copy()
summary_sgd_under_imblearn_extended.loc[len(summary_sgd_under_imblearn_extended.index)] = ['AP', average_precision]
summary_sgd_under_imblearn_extended.set_index('Metric')

summary_sgd_under_imblearn_index = summary_sgd_under_imblearn_extended.T
summary_sgd_under_imblearn_index.columns = summary_sgd_under_imblearn_index.iloc[0]
summary_sgd_under_imblearn_index.drop(summary_sgd_under_imblearn_index.index[0], inplace = True)
summary_sgd_under_imblearn_index

## Random over-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(sgd, X_train_over_imblearn_scaled_minmax, y_train_over_imblearn, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_sgd_over_imblearn = summary
summary_sgd_over_imblearn.set_index('Metric')

y_score = sgd.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_sgd_over_imblearn_extended = summary.copy()
summary_sgd_over_imblearn_extended.loc[len(summary_sgd_over_imblearn_extended.index)] = ['AP', average_precision]
summary_sgd_over_imblearn_extended.set_index('Metric')

summary_sgd_over_imblearn_index = summary_sgd_over_imblearn_extended.T
summary_sgd_over_imblearn_index.columns = summary_sgd_over_imblearn_index.iloc[0]
summary_sgd_over_imblearn_index.drop(summary_sgd_over_imblearn_index.index[0], inplace = True)
summary_sgd_over_imblearn_index

## Synthetic minority over-sampling technique (SMOTE)

In [ ]:
# Elements of confusion matrix

classification(sgd, X_train_over_smote_scaled_minmax, y_train_over_smote, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_sgd_over_smote = summary
summary_sgd_over_smote.set_index('Metric')

y_score = sgd.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_sgd_over_smote_extended = summary.copy()
summary_sgd_over_smote_extended.loc[len(summary_sgd_over_smote_extended.index)] = ['AP', average_precision]
summary_sgd_over_smote_extended.set_index('Metric')

summary_sgd_over_smote_index = summary_sgd_over_smote_extended.T
summary_sgd_over_smote_index.columns = summary_sgd_over_smote_index.iloc[0]
summary_sgd_over_smote_index.drop(summary_sgd_over_smote_index.index[0], inplace = True)
summary_sgd_over_smote_index

## Under-sampling via NearMiss

In [ ]:
# Elements of confusion matrix

classification(sgd, X_train_under_nm_scaled_minmax, y_train_under_nm, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_sgd_under_nm = summary
summary_sgd_under_nm.set_index('Metric')

y_score = sgd.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_sgd_under_nm_extended = summary.copy()
summary_sgd_under_nm_extended.loc[len(summary_sgd_under_nm_extended.index)] = ['AP', average_precision]
summary_sgd_under_nm_extended.set_index('Metric')

summary_sgd_under_nm_index = summary_sgd_under_nm_extended.T
summary_sgd_under_nm_index.columns = summary_sgd_under_nm_index.iloc[0]
summary_sgd_under_nm_index.drop(summary_sgd_under_nm_index.index[0], inplace = True)
summary_sgd_under_nm_index

## Summary of SGD models

In [ ]:
summary_sgd = pd.DataFrame(columns = ['Metric'])

summary_sgd['Metric'] = EvalMetricLabels
summary_sgd_list = [summary_sgd_unaltered, summary_sgd_under, summary_sgd_over, summary_sgd_under_imblearn,
                    summary_sgd_over_imblearn, summary_sgd_over_smote, summary_sgd_under_nm]

for i in summary_sgd_list:
    summary_sgd = pd.merge(summary_sgd, i, on = 'Metric')
    
TrainingSetsMetric = TrainingSets.copy()
TrainingSetsMetric.insert(0, 'Metric')

summary_sgd.columns = TrainingSetsMetric
summary_sgd.set_index('Metric', inplace = True)
summary_sgd

In [ ]:
# Visual comparison of the model applied on different training sets through various evaluation metrics

summary_visual(summary_sgd)

<a name='13.-Ridge-Classifier'></a>
# 13. Ridge Classifier

In [ ]:
ridge = RidgeClassifier()

We use normalised features as the ridge classifier employs $l^2$ regularization through an additive penalty term in the objective function.

## Unaltered training set

In [ ]:
# Elements of confusion matrix

classification(ridge, X_train_scaled_minmax, y_train, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_ridge_unaltered = summary.copy()
summary_ridge_unaltered.set_index('Metric')

y_score = ridge.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_ridge_unaltered_extended = summary.copy()
summary_ridge_unaltered_extended.loc[len(summary_ridge_unaltered_extended.index)] = ['AP', average_precision]
summary_ridge_unaltered_extended.set_index('Metric')

summary_ridge_unaltered_index = summary_ridge_unaltered_extended.T
summary_ridge_unaltered_index.columns = summary_ridge_unaltered_index.iloc[0]
summary_ridge_unaltered_index.drop(summary_ridge_unaltered_index.index[0], inplace = True)
summary_ridge_unaltered_index

## Random under-sampling

In [ ]:
# Elements of confusion matrix

classification(ridge, X_train_under_scaled_minmax, y_train_under, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_ridge_under = summary.copy()
summary_ridge_under.set_index('Metric')

y_score = ridge.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_ridge_under_extended = summary.copy()
summary_ridge_under_extended.loc[len(summary_ridge_under_extended.index)] = ['AP', average_precision]
summary_ridge_under_extended.set_index('Metric')

summary_ridge_under_index = summary_ridge_under_extended.T
summary_ridge_under_index.columns = summary_ridge_under_index.iloc[0]
summary_ridge_under_index.drop(summary_ridge_under_index.index[0], inplace = True)
summary_ridge_under_index

## Random over-sampling

In [ ]:
# Elements of confusion matrix

classification(ridge, X_train_over_scaled_minmax, y_train_over, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_ridge_over = summary.copy()
summary_ridge_over.set_index('Metric')

y_score = ridge.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_ridge_over_extended = summary.copy()
summary_ridge_over_extended.loc[len(summary_ridge_over_extended.index)] = ['AP', average_precision]
summary_ridge_over_extended.set_index('Metric')

summary_ridge_over_index = summary_ridge_over_extended.T
summary_ridge_over_index.columns = summary_ridge_over_index.iloc[0]
summary_ridge_over_index.drop(summary_ridge_over_index.index[0], inplace = True)
summary_ridge_over_index

## Random under-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(ridge, X_train_under_imblearn_scaled_minmax, y_train_under_imblearn, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_ridge_under_imblearn = summary.copy()
summary_ridge_under_imblearn.set_index('Metric')

y_score = ridge.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_ridge_under_imblearn_extended = summary.copy()
summary_ridge_under_imblearn_extended.loc[len(summary_ridge_under_imblearn_extended.index)] = ['AP', average_precision]
summary_ridge_under_imblearn_extended.set_index('Metric')

summary_ridge_under_imblearn_index = summary_ridge_under_imblearn_extended.T
summary_ridge_under_imblearn_index.columns = summary_ridge_under_imblearn_index.iloc[0]
summary_ridge_under_imblearn_index.drop(summary_ridge_under_imblearn_index.index[0], inplace = True)
summary_ridge_under_imblearn_index

## Random over-sampling with imbalanced-learn library

In [ ]:
# Elements of confusion matrix

classification(ridge, X_train_over_imblearn_scaled_minmax, y_train_over_imblearn, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_ridge_over_imblearn = summary.copy()
summary_ridge_over_imblearn.set_index('Metric')

y_score = ridge.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_ridge_over_imblearn_extended = summary.copy()
summary_ridge_over_imblearn_extended.loc[len(summary_ridge_over_imblearn_extended.index)] = ['AP', average_precision]
summary_ridge_over_imblearn_extended.set_index('Metric')

summary_ridge_over_imblearn_index = summary_ridge_over_imblearn_extended.T
summary_ridge_over_imblearn_index.columns = summary_ridge_over_imblearn_index.iloc[0]
summary_ridge_over_imblearn_index.drop(summary_ridge_over_imblearn_index.index[0], inplace = True)
summary_ridge_over_imblearn_index

## Synthetic minority over-sampling technique (SMOTE)

In [ ]:
# Elements of confusion matrix

classification(ridge, X_train_over_smote_scaled_minmax, y_train_over_smote, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_ridge_over_smote = summary.copy()
summary_ridge_over_smote.set_index('Metric')

y_score = ridge.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_ridge_over_smote_extended = summary.copy()
summary_ridge_over_smote_extended.loc[len(summary_ridge_over_smote_extended.index)] = ['AP', average_precision]
summary_ridge_over_smote_extended.set_index('Metric')

summary_ridge_over_smote_index = summary_ridge_over_smote_extended.T
summary_ridge_over_smote_index.columns = summary_ridge_over_smote_index.iloc[0]
summary_ridge_over_smote_index.drop(summary_ridge_over_smote_index.index[0], inplace = True)
summary_ridge_over_smote_index

## Under-sampling via NearMiss

In [ ]:
# Elements of confusion matrix

classification(ridge, X_train_under_nm_scaled_minmax, y_train_under_nm, X_test_scaled_minmax, y_test)

# Summary of evaluation metrics

summary_ridge_under_nm = summary.copy()
summary_ridge_under_nm.set_index('Metric')

y_score = ridge.decision_function(X_test)
average_precision = average_precision_score(y_test, y_score)

summary_ridge_under_nm_extended = summary.copy()
summary_ridge_under_nm_extended.loc[len(summary_ridge_under_nm_extended.index)] = ['AP', average_precision]
summary_ridge_under_nm_extended.set_index('Metric')

summary_ridge_under_nm_index = summary_ridge_under_nm_extended.T
summary_ridge_under_nm_index.columns = summary_ridge_under_nm_index.iloc[0]
summary_ridge_under_nm_index.drop(summary_ridge_under_nm_index.index[0], inplace = True)
summary_ridge_under_nm_index

## Summary of ridge classifiers

In [ ]:
summary_ridge = pd.DataFrame(columns = ['Metric'])

summary_ridge['Metric'] = EvalMetricLabels
summary_ridge_list = [summary_ridge_unaltered, summary_ridge_under, summary_ridge_over, summary_ridge_under_imblearn,
                      summary_ridge_over_imblearn, summary_ridge_over_smote, summary_ridge_under_nm]

for i in summary_ridge_list:
    summary_ridge = pd.merge(summary_ridge, i, on = 'Metric')
    
TrainingSetsMetric = TrainingSets.copy()
TrainingSetsMetric.insert(0, 'Metric')

summary_ridge.columns = TrainingSetsMetric
summary_ridge.set_index('Metric', inplace = True)
summary_ridge

In [ ]:
# Visual comparison of the model applied on different training sets through various evaluation metrics

summary_visual(summary_ridge)

<a name='14.-Conclusion'></a>
# 14. Conclusion

We choose the training set for each model on which it performs best and tabulate their performance in terms of **F2-Score**, which considers the facts that the dataset is imbalanced, the positive class (fraudulent transactions) is more important than the negative class (authentic transactions) and also that false negatives are more costly than false positives. Additionally, we report **MCC** (captures all-round performance across classes) and **Recall** (focuses only on the crucial postive class).

In [ ]:
# Comparison of classification models

"""
In the final table, models are sorted in decreasing order of their performance on the testing set, measured in F2-Score

The training set which is fed to a classifier is mentioned in parenthesis following the name of that classifier

Unaltered: unaltered training set
ROS-IL: random over-sampling of minority class via imbalanced-learn library
SMOTE: Over-sampling of minority class via synthetic minority over-sampling technique (SMOTE)
"""

models = ['Logistic Regression (Unaltered)', 'KNN (Unaltered)', 'Decision Tree (ROS-IL)',
          'Linear SVM (Unaltered)', 'Naive Bayes (SMOTE)', 'Random Forest (SMOTE)',
          'LDA (Unaltered)', 'SGD (Unaltered)', 'Ridge Classifier (Unaltered)']
metrics = ['F2-Score', 'MCC', 'Recall']
cols = ['Classification model'] + metrics

model_comparison = pd.DataFrame(columns = cols)
model_comparison['Classification model'] = models

summary_list = [summary_logreg_unaltered_index, summary_knn_unaltered_index, summary_dt_over_imblearn_index,
                summary_svm_linear_unaltered_index, summary_nb_over_smote_index, summary_rf_over_smote_index,
                summary_lda_unaltered_index, summary_sgd_unaltered_index, summary_ridge_unaltered_index]

F2_score = []
MCC = []
Recall = []

for i in summary_list:
  F2_score.append(float(i['F2-Score']))
  MCC.append(float(i['MCC']))
  Recall.append(float(i['Recall']))

model_comparison['F2-Score'] = F2_score
model_comparison['MCC'] = MCC
model_comparison['Recall'] = Recall

model_comparison.set_index('Classification model', inplace = True)
model_comparison_descending_F2 = model_comparison.sort_values(by = ['F2-Score'], ascending = False)
model_comparison_descending_F2

The **Random Forest** algorithm applied on the training set obtained after oversampling the minority class (fraudulent transactions) via **SMOTE** appears to be the best classification model for the problem at hand.

**SMOTE** is one of the best choices to oversample the minority class when the data is imbalanced. It is not surprising that **Random Forest** turns out to be one of the most suitable classifiers for the problem due to the following reasons:

- The algorithm works well in dealing with large datasets with high dimensions.

- It is less affected by the presence of outliers in feature variables compared to other algorithms.

- It does not make any distributional assumption on the feature variables.

- It handles collinearity (linear dependence among features) implicitly.

- It automatically ignores the features which are not useful, effectively doing feature selection on its own.